In [1]:
# First version: 18th of March 2022
# Author: Evangelos Papoutsellis
# Copyright 2022 Science and Techonology Facilities Council

# This software was developed during the Math+ “Maths meets Image” hackathon 2022.

# Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless
# required by applicable law or agreed to in writing, software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License
# for the specific language governing permissions and limitations under the License.


from cil.utilities.noise import gaussian
from cil.utilities.display import show2D
from cil.optimisation.operators import FiniteDifferenceOperator, GradientOperator, BlockOperator
from cil.optimisation.functions import L1Norm, MixedL21Norm, L2NormSquared, BlockFunction
from cil.optimisation.algorithms import PDHG
from cil.utilities.jupyter import islicer
from cil.framework import ImageGeometry

import nibabel as nib
import os
import numpy as np


# Dynamic denoising with isotropic-spatial TV coupled anisotropic with temporal TV

$$ \underset{u}{\mathrm{argmin}} \big\{\frac{1}{2}\| u - g \|^{2} + \alpha\|\partial_{t} u\|_{1} + \beta\|\nabla u\|_{2} \big\}$$


$$ \underset{u}{\mathrm{argmin}} \mathcal{F}(Ku) + \mathcal{G}(u)$$


### Algorithm used :  Primal-Dual Hybrid algorithm

In [2]:
def plot_2d_image(idx,vol,title,clims=None,cmap="viridis"):
    """Customized version of subplot to plot 2D image"""
    plt.subplot(*idx)
    plt.imshow(vol,cmap=cmap)
    if not clims is None:
        plt.clim(clims)
    plt.colorbar()
    plt.title(title)
    plt.axis("off")


def crop_and_fill(templ_im, vol):
    """Crop volumetric image data and replace image content in template image object"""
    # Get size of template image and crop
    idim_orig = templ_im.as_array().shape
    idim = (1,)*(3-len(idim_orig)) + idim_orig
    offset = (numpy.array(vol.shape) - numpy.array(idim)) // 2
    vol = vol[offset[0]:offset[0]+idim[0], offset[1]:offset[1]+idim[1], offset[2]:offset[2]+idim[2]]
    
    # Make a copy of the template to ensure we do not overwrite it
    templ_im_out = templ_im.copy()
    
    # Fill image content 
    templ_im_out.fill(numpy.reshape(vol, idim_orig))
    return(templ_im_out)

In [3]:

data_path = '/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/NOR/patient071/'
example_ni1 = os.path.join(data_path, 'image.nii.gz')
n1_img = nib.load(example_ni1).get_fdata() #get a numpy


In [4]:
dynamic_img = n1_img[:,:,5,0,:,0] # what are the last 3 slices???
print(" Shape is {}".format(dynamic_img.shape))

 Shape is (192, 256, 30)


### Make a cil object, Add gaussian noise

In [5]:

ig = ImageGeometry(voxel_num_x = dynamic_img.shape[1], 
                   voxel_num_y = dynamic_img.shape[0], 
                   channels = 30)
tmp_data = ig.allocate()
tmp_data.fill(np.moveaxis(dynamic_img, 2, 0)) # change order of axis
tmp_data_res = tmp_data/tmp_data.max() 
noisy_data = gaussian(tmp_data_res, seed=10, var = 0.001) # add noise gaussian

In [6]:
islicer(tmp_data.array, direction=0)

interactive(children=(IntSlider(value=15, continuous_update=False, description='X', max=29), FloatRangeSlider(…

IntSlider(value=15, continuous_update=False, description='X', max=29)

In [7]:
L1Norm.convex_conjugate??

Signature: L1Norm.convex_conjugate(self, x)
Docstring:
Returns the value of the convex conjugate of the L1Norm function at x.
Here, we need to use the convex conjugate of L1Norm, which is the Indicator of the unit 
:math:`L^{\infty}` norm

Consider the following cases:
        
        a) .. math:: F^{*}(x^{*}) = \mathbb{I}_{\{\|\cdot\|_{\infty}\leq1\}}(x^{*}) 
        b) .. math:: F^{*}(x^{*}) = \mathbb{I}_{\{\|\cdot\|_{\infty}\leq1\}}(x^{*}) + <x^{*},b>      


.. math:: \mathbb{I}_{\{\|\cdot\|_{\infty}\leq1\}}(x^{*}) 
    = \begin{cases} 
    0, \mbox{if } \|x^{*}\|_{\infty}\leq1\\
    \infty, \mbox{otherwise}
    \end{cases}
Source:   
    def convex_conjugate(self,x):
        
        r"""Returns the value of the convex conjugate of the L1Norm function at x.
        Here, we need to use the convex conjugate of L1Norm, which is the Indicator of the unit 
        :math:`L^{\infty}` norm
        
        Consider the following cases:
                
                a) .. math:: F^{*

In [12]:
alpha = 0.001
beta = 0.1
Dt = FiniteDifferenceOperator(ig, direction=0) # time is first
Grad = GradientOperator(ig, correlation="Space")

print(" Range of Dt = {}".format(Dt.range.shape))
print(" Range of Grad = {}".format(Grad.range.shape))

f1 = alpha * L1Norm()
f2 = beta * MixedL21Norm()

# return 0 for the Indicator function, avoid inf
from cil.framework import BlockDataContainer
def convex_conjugate_l21(x):

    if not isinstance(x, BlockDataContainer):
        raise ValueError('__call__ expected BlockDataContainer, got {}'.format(type(x))) 

    return 0.
    
def convex_conjugate_l1(x):

     return 0.

f1.convex_conjugate = convex_conjugate_l1
f2.convex_conjugate = convex_conjugate_l21


K = BlockOperator(Dt, Grad)
F = BlockFunction(f1, f2)
G = L2NormSquared(b = noisy_data)

Initialised GradientOperator with numpy backend
 Range of Dt = (30, 192, 256)
 Range of Grad = (2, 1)


In [13]:
pdhg = PDHG(f=F, g=G, operator=K, max_iteration=100, 
            update_objective_interval=10, check_convergence=False)
pdhg.run(verbose=2)

PDHG setting up
PDHG configured
     Iter   Max Iter     Time/Iter        Primal          Dual     Primal-Dual
                               [s]     Objective     Objective             Gap
        0        100         0.000    2.03755e+04  -0.00000e+00    2.03755e+04


/opt/SIRF-SuperBuild/INSTALL/python/cil/framework/framework.py:2381: RuntimeWarning: invalid value encountered in true_divide
  pwop(self.as_array(), x2.as_array(), *args, **kwargs )


       10        100         0.127    2.90766e+03   2.43577e+03    4.71891e+02
       20        100         0.118    2.75287e+03   2.53324e+03    2.19629e+02
       30        100         0.115    2.70975e+03   2.56602e+03    1.43736e+02
       40        100         0.114    2.68928e+03   2.58287e+03    1.06407e+02
       50        100         0.108    2.67708e+03   2.59316e+03    8.39197e+01
       60        100         0.103    2.66890e+03   2.60004e+03    6.88533e+01
       70        100         0.099    2.66302e+03   2.60495e+03    5.80795e+01
       80        100         0.096    2.65861e+03   2.60859e+03    5.00249e+01
       90        100         0.094    2.65519e+03   2.61139e+03    4.38028e+01
      100        100         0.093    2.65247e+03   2.61361e+03    3.88642e+01
----------------------------------------------------------------------------
      100        100         0.093    2.65247e+03   2.61361e+03    3.88642e+01
Stop criterion has been reached.



In [ ]:
sol = pdhg.solution
islicer(sol, direction=0)

# Dynamic denoising with anisotropic spatiotemporal TV, i.e. uncouple all dims.

$$ \underset{u}{\mathrm{argmin}} \big\{\| u - g \|^{2} + \alpha ( \|\partial_{t} u\|_{1} + \|\partial_{y} u\|_{1} + \|\partial_{x} u\|_{1})$$


In [ ]:
Dt = FiniteDifferenceOperator(ig, direction=0) 
Dy = FiniteDifferenceOperator(ig, direction=1) 
Dx = FiniteDifferenceOperator(ig, direction=2) 
K = BlockOperator(Dt, Dy, Dx)

# avoid inf values, dual objective
def convex_conjugate(self,x):

    tmp = x.abs().max()
    if tmp<=1.+1e-4:            
        if self.b is not None:
            return self.b.dot(x)
        else:
            return 0.
    return np.inf

L1Norm.convex_conjugate = convex_conjugate
f1 = L1Norm()
f2 = L1Norm()
f3 = L1Norm()

alpha = 0.5
F = alpha * BlockFunction(f1, f2, f3)
G = L2NormSquared(b=noisy_data)

pdhg_uncouple = PDHG(f=F, g=G, operator=K, max_iteration=100, 
            update_objective_interval=10, check_convergence=False)
pdhg_uncouple.run(verbose=2)




In [ ]:
sol = pdhg_uncouple.solution
islicer(sol, direction=0)